**Penerapan Algoritma _Clustering_ untuk Pengelompokkan Saham IDX Berdasarkan Indikator-indikator Fundamental - Submission Machine Learning Terapan Dicoding** 

oleh: Fikri Septrian Anggara (fikri_anggara_2c3r)

### Import library yang diperlukan

In [29]:
# untuk pengolahan data
import pandas as pd

# untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# untuk pembangunan klaster
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

# untuk visualisasi klaster
from scipy.cluster.hierarchy import dendrogram, linkage

# untuk scaling data
from sklearn.preprocessing import MinMaxScaler

# untuk menghitung jarak
from scipy.spatial.distance import pdist, cdist

# untuk menghitung evaluasi klaster
from sklearn.metrics import silhouette_score

# untuk melakukan reduksi dimensi
from sklearn.decomposition import PCA

### Menyiapkan Dataset
dataset diperoleh dari kaggle, terdapat dua dataset yang digunakan yaitu:
- [financial statement idx stocks](https://www.kaggle.com/datasets/greegtitan/financial-statement-idx-stocks?resource=download) (kaggle)
 - [daftar saham](https://www.kaggle.com/datasets/muamkh/ihsgstockdata?select=DaftarSaham.csv) (kaggle)

In [30]:
# load data
stockQuarter = pd.read_csv('data/quarter.csv')
masterStock = pd.read_csv('data/DaftarSaham.csv')

### Overview Data

In [31]:
# banyaknya baris dan kolom (baris, kolom)
print(stockQuarter.shape)
print(masterStock.shape)

(208691, 8)
(829, 14)


In [32]:
# ambil sampel random sebanyak 5 data
stockQuarter.sample(n=5)


,symbol,account,type,2021-09-30,2021-12-31,2022-03-31,2022-06-30,2022-09-30
37067,BMSR,IS,Total Revenue,7.150737e+11,1.072165e+12,1.326827e+12,1.138574e+12,NaN
35381,BLTZ,CF,Preferred Stock Payments,NaN,NaN,NaN,NaN,NaN
26794,BCAP,CF,Cash Paymentsfor Loans,NaN,NaN,NaN,NaN,NaN
166435,ROTI,CF,Change In Dividend Payable,NaN,NaN,NaN,NaN,NaN
16580,ASLC,CF,Other Cash Paymentsfrom Operating Activities,NaN,NaN,-1.181278e+10,-2.103036e+10,NaN


terdapat banyak **null value** pada data saham perkuarter

In [33]:
# ambil sampel random sebanyak 5 data
masterStock.sample(n=5)

,Code,Name,ListingDate,Shares,ListingBoard,Sector,LastPrice,MarketCap,MinutesFirstAdded,MinutesLastUpdated,HourlyFirstAdded,HourlyLastUpdated,DailyFirstAdded,DailyLastUpdated
751,TIRA,Tira Austenite Tbk,1993-07-27,5.880000e+08,Pengembangan,Industrials,408.0,2.399040e+11,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2001-04-16,2023-01-06
46,ARII,Atlas Resources Tbk.,2011-11-08,3.131000e+09,Pengembangan,Energy,264.0,8.265840e+11,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2011-11-08,2023-01-06
240,DSNG,Dharma Satya Nusantara Tbk.,2013-06-14,1.059984e+10,Utama,Consumer Non-Cyclicals,640.0,6.783899e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2013-06-14,2023-01-06
179,CBMF,Cahaya Bintang Medan Tbk.,2020-04-09,1.875000e+09,Pengembangan,Consumer Cyclicals,50.0,9.375000e+10,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2020-04-09,2023-01-06
302,GOLD,Visi Telekomunikasi Infrastruk,2010-07-07,1.277276e+09,Pengembangan,Infrastructures,494.0,6.309743e+11,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2010-07-07,2023-01-06


In [45]:
# 5 number summary stock quarter
stockQuarter.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
symbol,208691,544,AALI,391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
account,208691,3,BS,73461,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,208149,388,Interest Received,1080,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-30,51722.0,NaN,NaN,NaN,1918202605901.322021,24798786684018.976562,-175313000000000.0,0.0,5837091260.0,233651664187.5,1637950171000000.0
2021-12-31,51538.0,NaN,NaN,NaN,2042450921973.788574,26106014954333.765625,-438813037000000.0,0.0,7402772575.0,257243562462.0,1725611128000000.0
2022-03-31,53449.0,NaN,NaN,NaN,1940242135220.246094,25585023607145.355469,-223695701000000.0,0.0,5526650874.0,214030861495.0,1734074740000000.0
2022-06-30,50375.0,NaN,NaN,NaN,2139261720062.068848,26835599887835.515625,-194169000000000.0,0.0,7726820540.0,265889690000.0,1785706841000000.0
2022-09-30,17013.0,NaN,NaN,NaN,3359261415683.166504,33720778198892.175781,-197134246000000.0,79886.0,20220521000.0,557420000000.0,1839336498000000.0


- terdapat **544** buah saham emiten
- terdapat **388** variabel pada laporan keuangan
- terdapat **3** kategori akun yaitu balance sheet, cash-flow dan income statement

In [44]:
# 5 number summary master stock
masterStock.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Code,829,829,AALI,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,829,829,Astra Agro Lestari Tbk.,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ListingDate,829,713,2022-11-08 00:00:00,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shares,829.0,NaN,NaN,NaN,10946944360.291918,45395931792.267258,3600000.0,1230839821.0,3128090000.0,9327465018.0,1184363929502.0
ListingBoard,829,3,Pengembangan,422,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sector,829,11,Consumer Cyclicals,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LastPrice,824.0,NaN,NaN,NaN,1363.453883,3733.432446,25.0,101.75,287.0,975.0,38000.0
MarketCap,824.0,NaN,NaN,NaN,10076631834955.722656,50531421356601.734375,9000000000.0,297528301025.0,1111304868160.0,4236952973400.0,1012951085850000.0
MinutesFirstAdded,787,60,2021-11-01 09:00:00,698,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MinutesLastUpdated,787,18,2023-01-06 15:59:00,759,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Terdapat 11 sektor pada master stok dengan **Consumer Cyclicals** adalah sektor yang paling banyak emitennya
- Terdapat 829 buah saham emiten

### Exploratory Data Analysis (EDA)
- pada data yang digunakan penulis hanya menggunakan data kuarter ke-3 2022 
variabel yang penulis gunakan untuk membangun klaster terdapat pada kolom `type` di data `stockQuarter`.
selain itu penulis juga ingin melihat pola pada tiap sektor, maka data `stockQuarter` perlu direshape

In [46]:
# variabel pada laporan keuangan
pd.DataFrame(pd.unique(stockQuarter['type']))

,0
0,Long Term Equity Investment
1,Additional Paid In Capital
2,Long Term Debt
3,Ordinary Shares Number
4,Total Assets
...,...
384,Interest Income From Deposits
385,Credit Losses Provision
386,Other Interest Income
387,Interest Expense For Federal Funds Sold And Se...


In [54]:
# temp = stockQuarter[['account', 'type']].copy()
# temp = temp.drop_duplicates()
# temp.to_csv('data/financialStatement.csv')

terdapat 389 variabel pada laporan keuangan

In [47]:
# buat kolom baru, gabungan antara account dan type
stockQuarter['account_type'] = stockQuarter['account']+'_'+stockQuarter['type']
stockQuarter.head()

,symbol,account,type,2021-09-30,2021-12-31,2022-03-31,2022-06-30,2022-09-30,account_type
0,AALI,BS,Long Term Equity Investment,3.235200e+11,3.309040e+11,3.275800e+11,2.850690e+11,4.714630e+11,BS_Long Term Equity Investment
1,AALI,BS,Additional Paid In Capital,3.878995e+12,3.878995e+12,3.878995e+12,3.878995e+12,3.878995e+12,BS_Additional Paid In Capital
2,AALI,BS,Long Term Debt,5.709887e+12,2.131944e+12,2.144732e+12,2.220370e+12,3.281008e+12,BS_Long Term Debt
3,AALI,BS,Ordinary Shares Number,1.924688e+09,1.924688e+09,1.924688e+09,1.924688e+09,1.924688e+09,BS_Ordinary Shares Number
4,AALI,BS,Total Assets,2.969401e+13,3.039991e+13,3.123278e+13,3.023399e+13,3.263865e+13,BS_Total Assets


### reshaping data

In [18]:
# ambil data 2021 saja
data2021 = stockQuarter[['symbol', 'account_type', '2022-06-30']].copy()
# reshaping data, kolom" data yang baru merupakan value dari 'type' dari df yang lama
dataReshaped = data2021.pivot_table(
    index='symbol',
    columns='account_type',
    values='2022-06-30'
).reset_index()
dataReshaped.head()

account_type,symbol,BS_Accounts Payable,BS_Accounts Receivable,BS_Accumulated Depreciation,BS_Additional Paid In Capital,BS_Allowance For Doubtful Accounts Receivable,BS_Allowance For Loans And Lease Losses,BS_Assets Held For Sale,BS_Available For Sale Securities,BS_Buildings And Improvements,...,IS_Selling General And Administration,IS_Special Income Charges,IS_Tax Effect Of Unusual Items,IS_Tax Provision,IS_Tax Rate For Calcs,IS_Total Premiums Earned,IS_Total Revenue,IS_Total Unusual Items,IS_Total Unusual Items Excluding Goodwill,IS_Write Off
0,AALI,8.259340e+11,4.305280e+11,-1.276215e+13,3.878995e+12,-2.553900e+10,NaN,NaN,NaN,4.876408e+12,...,1.818320e+11,2.423000e+09,7.975096e+08,1.628460e+11,0.0,NaN,4.383457e+12,2.423000e+09,2.423000e+09,NaN
1,ABBA,4.565925e+10,1.927453e+10,-1.616807e+11,-4.555371e+10,-6.364641e+10,NaN,NaN,1.659867e+11,2.361747e+10,...,4.808846e+09,0.000000e+00,0.000000e+00,3.356401e+08,0.0,NaN,3.270153e+10,0.000000e+00,0.000000e+00,0.0
2,ABDA,1.726138e+10,7.472096e+10,-1.051394e+11,8.109426e+09,NaN,NaN,NaN,-2.732850e+09,9.103365e+10,...,2.573074e+10,5.790000e+06,2.316000e+06,-7.352070e+08,0.0,1.570100e+11,1.933188e+11,5.790000e+06,5.790000e+06,NaN
3,ABMM,1.617792e+08,1.994225e+08,-7.378516e+08,1.150872e+08,-1.031222e+07,NaN,NaN,2.346308e+07,8.629613e+07,...,9.630067e+06,NaN,0.000000e+00,2.569592e+07,0.0,NaN,3.655084e+08,NaN,NaN,NaN
4,ACES,1.728831e+11,7.668345e+10,-1.918547e+12,4.405749e+11,5.312285e+06,NaN,NaN,NaN,2.333466e+12,...,8.751484e+10,-6.539966e+09,-1.060284e+09,1.740837e+10,0.0,NaN,1.681350e+12,-6.539966e+09,-6.539966e+09,-5312285.0


### saham yang tersedia

In [17]:
pd.DataFrame(pd.unique(dataReshaped['symbol']))

,0
0,AALI
1,ABBA
2,ABDA
3,ABMM
4,ACES
...,...
498,WSKT
499,WTON
500,YPAS
501,ZBRA


In [19]:
# cek null value dari masing masing kolom
nulltable = pd.DataFrame(dataReshaped.isnull().sum().reset_index().iloc[1:])
nulltable.columns = ['financial statement', 'sum of null']
nulltable

,financial statement,sum of null
1,BS_Accounts Payable,17
2,BS_Accounts Receivable,70
3,BS_Accumulated Depreciation,10
4,BS_Additional Paid In Capital,5
5,BS_Allowance For Doubtful Accounts Receivable,162
...,...,...
222,IS_Total Premiums Earned,482
223,IS_Total Revenue,14
224,IS_Total Unusual Items,182
225,IS_Total Unusual Items Excluding Goodwill,182


In [27]:
dataReshaped.isnull().sum()

account_type
symbol                                         0
BS_Accounts Payable                           17
BS_Accounts Receivable                        70
BS_Accumulated Depreciation                   10
BS_Additional Paid In Capital                  5
                                            ... 
IS_Total Premiums Earned                     482
IS_Total Revenue                              14
IS_Total Unusual Items                       182
IS_Total Unusual Items Excluding Goodwill    182
IS_Write Off                                 398
Length: 227, dtype: int64

In [20]:
usedColumns = [k for k in dataReshaped.columns.tolist() if 'BS_' in k][:15]


In [22]:
dataFinal = dataReshaped[usedColumns]
dataFinal

account_type,BS_Accounts Payable,BS_Accounts Receivable,BS_Accumulated Depreciation,BS_Additional Paid In Capital,BS_Allowance For Doubtful Accounts Receivable,BS_Allowance For Loans And Lease Losses,BS_Assets Held For Sale,BS_Available For Sale Securities,BS_Buildings And Improvements,BS_Capital Lease Obligations,BS_Capital Stock,BS_Cash And Cash Equivalents,BS_Cash And Due From Banks,BS_Cash Cash Equivalents And Federal Funds Sold,BS_Cash Equivalents
0,8.259340e+11,4.305280e+11,-1.276215e+13,3.878995e+12,-2.553900e+10,NaN,NaN,NaN,4.876408e+12,NaN,9.623440e+11,1.956801e+12,NaN,NaN,1.761361e+12
1,4.565925e+10,1.927453e+10,-1.616807e+11,-4.555371e+10,-6.364641e+10,NaN,NaN,1.659867e+11,2.361747e+10,6.395203e+09,3.935893e+11,1.694638e+11,NaN,NaN,5.499452e+10
2,1.726138e+10,7.472096e+10,-1.051394e+11,8.109426e+09,NaN,NaN,NaN,-2.732850e+09,9.103365e+10,NaN,1.933167e+11,1.163526e+11,NaN,NaN,NaN
3,1.617792e+08,1.994225e+08,-7.378516e+08,1.150872e+08,-1.031222e+07,NaN,NaN,2.346308e+07,8.629613e+07,6.676648e+07,1.465549e+08,2.917540e+08,NaN,NaN,5.879837e+07
4,1.728831e+11,7.668345e+10,-1.918547e+12,4.405749e+11,5.312285e+06,NaN,NaN,NaN,2.333466e+12,7.785399e+11,1.715000e+11,2.475947e+12,NaN,NaN,3.473473e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,7.007696e+12,1.501147e+13,-4.096478e+12,1.379202e+13,-2.903400e+12,NaN,NaN,8.829152e+11,1.591137e+12,5.338342e+10,2.880681e+12,1.110274e+13,NaN,NaN,4.580050e+11
499,2.239787e+12,1.119072e+12,-1.791698e+12,9.886334e+11,-9.098793e+10,NaN,NaN,NaN,3.351810e+11,5.064611e+10,8.715467e+11,7.552673e+11,NaN,NaN,2.620000e+11
500,1.161044e+10,8.208517e+10,-1.646578e+11,2.805402e+10,-8.574100e+08,NaN,NaN,NaN,6.509308e+10,1.123992e+08,6.680001e+10,8.831136e+08,NaN,NaN,NaN
501,5.400567e+11,6.963487e+11,-3.866211e+11,1.167524e+12,-3.877312e+09,NaN,NaN,NaN,1.430180e+11,2.815831e+10,2.671036e+11,2.758749e+10,NaN,NaN,NaN
